In [ ]:
import cv2
import os
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Base paths
BASE_PATH = '/content/drive/MyDrive/PCB_DATASET'
TEMPLATE_DIR = os.path.join(BASE_PATH, 'PCB_USED')
TEST_DIR = os.path.join(BASE_PATH, 'images', 'Missing_hole')
OUTPUT_DIR = os.path.join(BASE_PATH, 'PCB_USED', 'Missing_Hole')  # Fixed output folder

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === ADD THESE LINES TO FORCE FOLDER VISIBILITY IN DRIVE ===
print(f"Creating output folder: {OUTPUT_DIR}")
print(f"Folder exists after creation: {os.path.exists(OUTPUT_DIR)}")

Creating output folder: /content/drive/MyDrive/PCB_DATASET/PCB_USED/Missing_Hole
Folder exists after creation: True


In [ ]:
# Create a dummy file inside the folder to force Google Drive sync faster
dummy_path = os.path.join(OUTPUT_DIR, '_FOLDER_CREATED_BY_SCRIPT.txt')
with open(dummy_path, 'w') as f:
    f.write('This folder was created for saving Missing Hole defect patches.\n')
    f.write('If you see this file, the folder is successfully created and synced.\n')
print(f"Dummy file created to force Drive sync: {dummy_path}")
# ===============================================================

Dummy file created to force Drive sync: /content/drive/MyDrive/PCB_DATASET/PCB_USED/Missing_Hole/_FOLDER_CREATED_BY_SCRIPT.txt


In [ ]:
# Fixed patch size
PATCH_SIZE = 64
HALF_PATCH = PATCH_SIZE // 2

# Area limits for valid defects
MIN_AREA = 1
MAX_AREA = 300

In [ ]:
# Get all test images
test_images = [f for f in os.listdir(TEST_DIR) if f.lower().endswith('.jpg')]

patch_id = 0  # Global counter for saved patches

print(f"Found {len(test_images)} test images in Missing_hole folder. Starting processing...\n")

for idx, test_filename in enumerate(test_images):
    # Extract template ID: e.g., "01_missing_hole_07.jpg" -> "01.JPG"
    template_id = test_filename.split('_')[0] + '.JPG'
    template_path = os.path.join(TEMPLATE_DIR, template_id)

    if not os.path.exists(template_path):
        print(f"Template not found: {template_path} (for {test_filename}) -> Skipping")
        continue

    # Load images
    template_rgb = cv2.imread(template_path)
    test_rgb = cv2.imread(os.path.join(TEST_DIR, test_filename))

    # Convert to grayscale
    template_gray = cv2.cvtColor(template_rgb, cv2.COLOR_BGR2GRAY)
    test_gray = cv2.cvtColor(test_rgb, cv2.COLOR_BGR2GRAY)

    # Get template size and divide by 4 as in sample
    h_temp, w_temp = template_gray.shape
    a = h_temp // 4
    b = w_temp // 4
    target_size = (b, a)

    # Resize both to same size
    template_resized = cv2.resize(template_gray, target_size)
    test_resized = cv2.resize(test_gray, target_size)

    # Preprocessing: Gaussian blur
    blur_template = cv2.GaussianBlur(template_resized, (3, 3), 0)
    blur_test = cv2.GaussianBlur(test_resized, (3, 3), 0)

    # Adaptive thresholding
    thresh_template = cv2.adaptiveThreshold(blur_template, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                            cv2.THRESH_BINARY, 15, 5)
    thresh_test = cv2.adaptiveThreshold(blur_test, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                        cv2.THRESH_BINARY, 15, 5)

    # Subtraction
    diff = cv2.subtract(thresh_template, thresh_test)

    # Clean noise
    diff_clean = cv2.medianBlur(diff, 3)

    # Otsu's thresholding (as required in milestone)
    _, otsu_mask = cv2.threshold(diff_clean, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Contour detection
    contours, _ = cv2.findContours(otsu_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h_img, w_img = test_resized.shape

    defects_in_this_image = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if MIN_AREA < area < MAX_AREA:
            x, y, w, h = cv2.boundingRect(cnt)
            cx = x + w // 2
            cy = y + h // 2

            # Patch coordinates
            x0 = max(0, cx - HALF_PATCH)
            y0 = max(0, cy - HALF_PATCH)
            x1 = min(w_img, cx + HALF_PATCH)
            y1 = min(h_img, cy + HALF_PATCH)

            # Only save full 64x64 patches
            if (x1 - x0) == PATCH_SIZE and (y1 - y0) == PATCH_SIZE:
                patch = test_resized[y0:y1, x0:x1]
                patch_path = os.path.join(OUTPUT_DIR, f"missing_hole_defect_{patch_id:04d}.png")
                cv2.imwrite(patch_path, patch)
                patch_id += 1
                defects_in_this_image += 1

    # Progress update
    if (idx + 1) % 20 == 0 or (idx + 1) == len(test_images):
        print(f"Processed {idx + 1}/{len(test_images)} images | Patches saved so far: {patch_id}")

Found 115 test images in Missing_hole folder. Starting processing...

Processed 20/115 images | Patches saved so far: 105
Processed 40/115 images | Patches saved so far: 203
Processed 60/115 images | Patches saved so far: 301
Processed 80/115 images | Patches saved so far: 380
Processed 100/115 images | Patches saved so far: 435
Processed 115/115 images | Patches saved so far: 479
